# Beginner tutorial: Optical components and systems in Asterix

This notebook is based on the script `tuto_asterix_model.py` on commit `213d669bfd944533c7befe9800a73b9b3592ab38`. The biggest difference is that we will not be saving fits files to disk, instead we will plot figures interactively in this notebook.

This tutorial goes over the basic usage of optical systems in Asterix: how to create them, how to change them, how to access their properties. We will first start with a generic optical system and then set up a model of the THD2 bench.

There main explanations about all of this are given in the PDF documentation file so this notebook will cover the minimum with some code that runs.

In [ ]:
# Imports
import os
import matplotlib as mpl
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import Asterix_root
from Asterix.utils import read_parameter_file
from Asterix.optics import Pupil, Coronagraph, DeformableMirror, Testbed

In [ ]:
# Some setup for pretty plotting
mpl.rc('image', origin='lower',   # Put the origin in the lower left corner.
       interpolation=None)        # Do not interpolate between pixels in the display.

## Creating and loading a configuration file

Note how "configuration file" and "parameter file" will be used interchangably in the following.

The basic sestup for all code within Asterix (optical systems, DMs, loops, simulations, ...) is defined within structured configuration file with file extension `.ini`. The purpose of this is to be able to save out a copy of this file during each run so that its details can be looked up at a later time, and also to easily create new simulation setups that can be run just by plugging in a new configuration file with updated parameters.

An example of such a file is given with `Example_param_file.ini`, and the abstract structure of this configuration file is given by the template `Param_configspec.ini`.

The first thing we will do is to load the example parameter file.

In [ ]:
# Load the template parameter file
parameter_file_ex = Asterix_root + "Example_param_file.ini"
config = read_parameter_file(parameter_file_ex)

This function will make sure any new configuration file is written in the correct format and contains all the required parameters. Let us first see how to access the information inside it.

In [ ]:
# You can plain-print the loaded configuration:
print(config)

In [ ]:
# Print all the sections of the configuration file:
for keys in config:
    print(keys)

We will now assign each set of configuration parameters that we need in this tutorial a new variable.

In [ ]:
modelconfig = config["modelconfig"]
DMconfig = config["DMconfig"]
Coronaconfig = config["Coronaconfig"]
SIMUconfig = config["SIMUconfig"]

Each of these variables is a dictionary with setup parameters for our simulations.

In [ ]:
# Inpsect "modelconfig" parameters:
for key, value in modelconfig.items():
    print(f"{key}: {value}")

In [ ]:
# Inpsect "DMconfig" parameters:
for key, value in DMconfig.items():
    print(f"{key}: {value}")

In [ ]:
# Inpsect "Coronaconfig" parameters:
for key, value in Coronaconfig.items():
    print(f"{key}: {value}")

In [ ]:
# Inpsect "SIMUconfig" parameters:
for key, value in SIMUconfig.items():
    print(f"{key}: {value}")

## Data management

All data created by Asterix will be saved to a main folder whose path is defined with a variable in the input configfile. Since we will not be saving any data out in this tutorial, we are ignoring this.

## Simple optical system with a Roman pupil

We will now create a simple optical system and see how we can operate it and what outputs we can create.

Note that we can override parameters in the configuration variables at runtime. This will not change opticla systems that were already created from these variables. In our case, we didn't do anything yet, so we will start by defining a small pupil of 80 pixels across so that the propagations don't take too much time.

In [ ]:
# Update the pixels across the pupil
modelconfig.update({'diam_pup_in_pix': 80})

In order to create a full optical system, we first create all the individual components which we concatenate at the end to compose our full optical system.

### Pupil aperture

We start by defining a pupil. Its physical radius is defined by the variable `prad` in the configfile.

In [ ]:
# Create a pupil
pup_round = Pupil(modelconfig)

To plot the pupil, we need to access its attribute `pup`.

In [ ]:
# Plot the round pupil
plt.imshow(pup_round.pup, cmap='Greys_r')
plt.title(f"Round pupil with {2*pup_round.prad} pixels across.")

We can also create pupils from input files, like in the case of the Roman pupil.

In [ ]:
# Create a pupil object representing the Roman pupil. For this one, we use rebin
# to resize a .fits size at the prefered size so you need to choose a divisor of the .fits file 
# size (500) for diam_pup_in_pix parameter: [100, 125, 250, 500]
modelconfig.update({'diam_pup_in_pix': 100})
pup_roman = Pupil(modelconfig, PupType="RomanPup")

In [ ]:
# Plot the Roman pupil
plt.imshow(pup_roman.pup, cmap='Greys_r')
plt.title(f"Roman Space Telescope pupil")

These objects have a bunch of parameters that define them which can also be set at the time the object is instantiated. For examples, we can create pupils with different sizes in pixels if we would like to, wehich can be useful for Lyot stops.

In [ ]:
# Create a round pupil with 200 pixels across
pup_round_100 = Pupil(modelconfig, prad=100)

# Plot this pupil
plt.imshow(pup_round_100.pup, cmap='Greys_r')
plt.title(f"Round pupil with 200 pixels across.")

Watch out how some of these parameters play together when you change only one of them. In the above example, we adjusted the number of pixels across the clear pupil, but not the number of pixels in the total array.

### Propagations through an optical element

Each of the pupil objects defined above has methods that perform the optical propagations (this is true for each optical element).

The first thing we can do is to calculate the electrical field (EF) right after the optical element in question, here shown for the Roman pupil. The result is a complex array, although in the case of a non-aberrated input wavefront at the plane of the pupil, this is just a real-valued array.

In [ ]:
# Calculate the EF right after the Roman pupil
EF_through_roman = pup_roman.EF_through(entrance_EF=1.)

In [ ]:
plt.imshow(EF_through_roman)
plt.title('Complex array which is real with perfect incoming wavefront.')

We can also calculate the associated PSF of this optical element (which is now a simple optical system) by using the method `todetector_intensity()`.

In [ ]:
psf_roman = pup_roman.todetector_intensity()

In [ ]:
plt.imshow(psf_roman, cmap='inferno', norm=LogNorm())
plt.title('Intensity of the Fourier transform of the Roman pupil')
plt.colorbar()

In [ ]:
# The chromaticity of the source is defined in all opitcal systems with three parameters:
print("Central wavelength: ", pup_roman.wavelength_0)
print("Bandwidth: ", pup_roman.Delta_wav)
print("Number of sub-wavelengths: ", pup_roman.nb_wav)

Also, all OpticalSystem objects have a transmission, which is the ratio of flux after the system, compared to a clear aperture of equivalent radius.

In [ ]:
# Show the transmission of the Roman pupil
print(f"Transmission of the Roman pupil: {pup_roman.transmission()}")

## Coronagraph

A coronagraph is a system composed of 3 planes. An apodization plane (PP), a FPM (FP) and a Lyot stop (PP). The coronagraph currently in the example configuration file does not have an apodization pupil because there is no such plane on the THD2 bench, but we can put one in, which is what we do with RoundPup below.

In [ ]:
# Define a round pupil in the apodization plane
Coronaconfig.update({'filename_instr_apod': "RoundPup"})

With this, we can create an optical system with a coronagraph from the default parameters in the configuration file.

In [ ]:
# Create the coronagraph
corono = Coronagraph(modelconfig, Coronaconfig)

For the coronagraph, we can measure 2 types of PSFs: with or without the FPM in the beam.

In [ ]:
# PSF without FPM
direct_psf = corono.todetector_intensity(center_on_pixel=True, noFPM=True)
# Get normalization factor from direct PSF
max_psf = direct_psf.max()

In [ ]:
# Coronagraphic PSF (with FPM)
coro_psf = corono.todetector_intensity(center_on_pixel=True, noFPM=False)

In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.imshow(direct_psf/max_psf, cmap='inferno', norm=LogNorm())
plt.title('Direct PSF')
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(coro_psf/max_psf, cmap='inferno')
plt.title('Coronagraphic PSF - perfect coronagraph')
plt.colorbar()

Note how the coronagraphic PSF here gives an almost empty array since the default coronagraph in the example parameterfile is a *perfect* coronagraph.

## Aberrations

We can create phase aberrations within a coronagraph by creating a phase screen with specific parameters, passed again from our input configuration file.

In [ ]:
# Create a phase screen
phase = corono.generate_phase_aberr(SIMUconfig)
print(type(phase))

In [ ]:
# Plot the phase screen
plt.imshow(phase, cmap='RdBu')
plt.title('Default phase screen')
plt.colorbar()

We can generate an E-field at the entrance of the coronagraph that includes this phase.

In [ ]:
aberrated_EF = corono.EF_from_phase_and_ampl(phase_abb=phase)

In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.imshow(np.abs(aberrated_EF)**2, cmap='inferno')
plt.title('EF intensity')
plt.colorbar()

plt.subplot(1,2,2)
plt.title('EF phase')
plt.imshow(np.angle(aberrated_EF), cmap='RdBu')
plt.colorbar()

We can use this E-field as an input wavefront to our coronagraph and propagate it through it - we can choose whether we calculate the output intensity directly or if we want to return the whole E-field that results at the end of the propagation.

In [ ]:
# Calculate coronagraphic PSF with chosen in put E-field
coro_psf_aber = corono.todetector_intensity(entrance_EF=aberrated_EF)

plt.imshow(coro_psf_aber/max_psf, cmap='inferno', norm=LogNorm())
plt.title('Coronagraphic PSF with phase aberrations')
plt.colorbar()

## Deformable mirrors

Deformable mirrors (DMs) can be in a pupil plane or outside a pupil plane. In the default configuration file, the DM we can create is in a pupil plane.

In [ ]:
# Create in-pupil DM
DM3 = DeformableMirror(modelconfig, DMconfig, Name_DM='DM3', Model_local_dir='temp')

DMs are also optical systems with the same propagation methods like we saw above, but with some extra parameters, for example `DMphase`. This lets us introduce a phase in the plane of the DM in question.

In [ ]:
# Propagate the EF "through" the DM
EF_though_DM = DM3.EF_through(entrance_EF=1., DMphase=0.)
print(type(EF_though_DM))
print(EF_though_DM)

What we encounter here is that the calculated E-field contains elements of only one value, which is why the return is collapsed into a single float.

In a more involved example, we can chose to inject the phase screen from above in the DM3 plane and repeat the propagation. In this case, an array will be returned.

In [ ]:
# Propagate "through" DM including a phase aberration
EF_though_DM_aber = DM3.EF_through(entrance_EF=aberrated_EF, DMphase=phase)
print(type(EF_though_DM_aber))
print(EF_though_DM_aber.shape)

In [ ]:
# Plot components of the E-field on this DM
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(np.abs(EF_though_DM_aber)**2, cmap='inferno')
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(np.angle(EF_though_DM_aber), cmap='RdBu')
plt.colorbar()

## Full optical system concatenation

Now that we have all these Optical Systems defined, we can play with them and concatenate them. The concatenation function takes 2 parameters:

- A list of Optical Systems
- A list of the same size containing the names of those systems so that you can access them

The list order is from the first optical system to the last in the path of the light (usually from entrance pupil to Lyot pupil).

In [ ]:
# Concatenate the round pupil, DM3 and coronagraph from above into a single optical system
testbed_1DM = Testbed([pup_round_100, DM3, corono],
                            ["entrancepupil", "DM3", "corono"])

Each of the subsystems can now be accessed individually with the name you have given it:  
--> testbed_1DM.entrancepupil, testbed.DM3, etc

To avoid any confusion in case of multiple DMs, the command to access DMs is now `XXXphase`, where `XXX` is the name of the DM, for example `DM3` or `DM1`.


In [ ]:
# Calculate the PSF through the whole optical system
psf_after_testbed = testbed_1DM.todetector_intensity(entrance_EF=aberrated_EF, DM3phase=phase)

plt.imshow(psf_after_testbed, cmap='inferno', norm=LogNorm())
plt.title('Full testbed PSF (unnormalized) - round pupil')
plt.colorbar()

We can now play with all the things we defined up to now, for example creating a 1DM-testbed with a Roman-like pupil mask.

In [ ]:
# Create testbed with RST pupil
testbed_1DM_romanpup = Testbed([pup_roman, DM3, corono],
                                     ["entrancepupil", "DM3", "corono"])
psf_after_roman_testbed = testbed_1DM_romanpup.todetector_intensity(entrance_EF=aberrated_EF, DM3phase=phase)

plt.imshow(psf_after_roman_testbed, cmap='inferno', norm=LogNorm())
plt.title('Full testbed PSF (unnormalized) - RST pupil')
plt.colorbar()

In [ ]:
# If you have DMs in your system, these are saved in the structure so that you can access them:
print("Number of DMs in testbed_1DM_romanpup:", testbed_1DM_romanpup.number_DMs)
print("Name of the DMs: ", testbed_1DM_romanpup.name_of_DMs)

## Simulating the THD2 testbed

If we want to define exactly the THD2 testbed, we need to add a second DM outside the pupil plane. This can take som time to initialize exactly because the first DM is outside a pupil plane.

In [ ]:
# We need to increase the number of pixels in the pupil if we add another DM.
modelconfig.update({'diam_pup_in_pix': 200})

Once we change the `modelconfig` secion of the configuration file/object, all the previously defined systems are of the wrong dimensions so they cannot be concatenated and must be recalculated.

In [ ]:
# Instantiate new pupil object from updated parameters
pup_round_larger = Pupil(modelconfig)

plt.imshow(pup_round_larger.pup, cmap='Greys_r')
plt.title("Slightly larger round pupil")

In [ ]:
# Create in-pupil DM
DM3 = DeformableMirror(modelconfig,
                              DMconfig,
                              Name_DM='DM3',
                              Model_local_dir='temp')

# Create out-of-pupil DM
DMconfig.update({'DM1_active': True})
DM1 = DeformableMirror(modelconfig,
                              DMconfig,
                              Name_DM='DM1',
                              Model_local_dir='temp')

In [ ]:
# We also need to "clear" the apodizer plane because  there is no apodizer plane on the THD2 bench.
Coronaconfig.update({'filename_instr_apod': "Clear"})
corono_thd = Coronagraph(modelconfig, Coronaconfig)

And then we concatenate all these components into our THD2 simulator.

In [ ]:
# And then just concatenate:
thd2 = Testbed([pup_round_larger, DM1, DM3, corono_thd],
                     ["entrancepupil", "DM1", "DM3", "corono"])

In [ ]:
# If you have DMs in your system, these are saved in the structure so that you can access them
print("Number of DMs on THD2:", thd2.number_DMs)
print("Name of the DMs: ", thd2.name_of_DMs)

And Now that we have all the tools, we can define even more complicated systems. Let's define a third DM, similar to DM1, but outside the pupil in the other dimension.

In [ ]:
DMconfig.update({'DM1_z_position': -15e-2})  # meter
DMconfig.update({'DM1_active': True})
DMnew = DeformableMirror(modelconfig,
                                DMconfig,
                                Name_DM='DM1',
                                Model_local_dir='temp')

The variable Name_DM in this function is to be understood as the type of DM you want to use (DM3 is a BMC32x32 type DM and DM1 is a BMC34x34) but the real name in the system is to be defined in the concatenation.

We also want to add a pupil in between all these DMs. Let's make it a round pupil for now, but we could imagine putting an apodizer here.

In [ ]:
# Instantiate pupil for in-between DMs
pupil_inbetween_DM = Pupil(modelconfig)

In [ ]:
# And a roman entrance pupil
pup_roman_larger = Pupil(modelconfig, PupType="RomanPup")

In [ ]:
# Let's concatenate everything!
testbed_3DM = Testbed([pup_roman_larger, DM1, DM3, pupil_inbetween_DM, DMnew, corono_thd],
                            ["entrancepupil", "DM1", "DM3", "pupil_inbetween_DM", "DMnew", "corono"])

In [ ]:
print("Number of DMs in testbed_3DM:", testbed_3DM.number_DMs)
print("Name of the DMs: ", testbed_3DM.name_of_DMs)